Continuing to review old questions from the stack.

Now cycling between 5 old and 5 new (but related problems).

1. Find largest palindrome in a string.
2. Number of unique decodings.

# 1. Find largest palindrome in a string.

Start from the 1st position. Keep a pointer ahead of the 1st pointer.
Iterate over all such positions, and keep checking for palilndromes.

A faster solution:
1. Given a string `s` which needs to be checked, check if `s[1:-1]` is a palindrome.
2. If it is, then `s` is a palindrome if `s[0]` and `s[-1]` are the same.
3. Build this structure iteratively, using uni-chars, then bi-chars, then tri-chars.
    * Solve for these problems first, then use these to check for bigger strings.

In [38]:
class LargestPalindrome:
    def slow(self, s):
        if len(s) == 1:
            return s
        
        ans = ''
        for i in range(len(s)):
            for j in range(i, len(s)):
                temp = s[i:j+1]
                if len(temp) > len(ans) and temp == temp[::-1]:
                    ans = temp
                    
        return ans
    
    def fast(self, s):
        structure = [[0 for i in range(len(s))] for j in range(len(s))]
        for i in range(len(s)):
            structure[i][i] = 1
            if (i+1) < len(s) and s[i] == s[i+1]:
                s[i][i+1] = 1
                
        ans = s[0:1]
        
        for gap in range(2, len(s)):
            for i in range(len(s)-gap):
                j = i + gap
                if structure[i+1][j-1] and s[i] == s[j]:
                    structure[i][j] = 1
                    temp = s[i:j+1]
                    if len(temp) > len(ans):
                        ans = temp
                        
        return ans

In [39]:
t = LargestPalindrome()
t.slow('abcba')

'abcba'

In [40]:
t.fast('abcban')

'abcba'

# 2. Find number of unique decodings for a string.

Given a string of numbers, convert it into all possible `decodings`.

Any number can be converted into an alphabet, except for 0, and substrings starting from 0.

```
s: 11106
1: 11, 10, 6
2: 1, 1, 10, 6
```

At each position, we can decode the current char, or the current+next chars.

In [56]:
from functools import lru_cache

class UniqueDecodings:
    def solve(self, s):
        self.ans = 0
        valid = set([str(i) for i in range(1, 27)])
        
        @lru_cache(maxsize=None)
        def decode(position):
            if position == len(s):
                return 1
            else:
                if s[position] in valid:
                    single = decode(position+1)
                    if (position+1) < len(s) and s[position:position+2] in valid:
                        double = decode(position+2)
                        return single + double
                    else:
                        return single
                else:
                    return 0
                        
        return decode(0)

In [58]:
t = UniqueDecodings()
t.solve('11106')

2